In [14]:
!pip install huggingface-hub -Uqq

In [16]:
from huggingface_hub import snapshot_download
from pathlib import Path

local_model_path = Path("./sentence2emb-model")
local_model_path.mkdir(exist_ok=True)
model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
commit_hash = "ef15aed8b328d308d7237b9bf15269f2cd19e268"

In [19]:
snapshot_download(repo_id=model_name, revision=commit_hash, cache_dir=local_model_path)

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]















Fetching 13 files:   8%|▊         | 1/13 [00:00<00:08,  1.39it/s]0<00:16, 65.4MB/s]





















































Fetching 13 files:  54%|█████▍    | 7/13 [00:14<00:12,  2.14s/it], 52.5MB/s]

































Fetching 13 files: 100%|██████████| 13/13 [00:28<00:00,  2.17s/it]


'sentence2emb-model/models--sentence-transformers--paraphrase-multilingual-mpnet-base-v2/snapshots/ef15aed8b328d308d7237b9bf15269f2cd19e268'

In [20]:
import sagemaker
from sagemaker import image_uris
import boto3
import os
import time
import json

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

In [32]:
local_model_path
local_model_path = Path("/QA-chatbot-workshop/sentence2emb-model")

In [34]:
s3_model_prefix = "LLM-RAG/workshop/sentence2emb-model"  # folder where model checkpoint will go
# model_snapshot_path = list(local_model_path.glob("**/snapshots/*"))[1]
s3_code_prefix = "LLM-RAG/workshop/sentence2emb_deploy_code"
s3_code_prefix

'LLM-RAG/workshop/sentence2emb_deploy_code'

In [35]:
model_snapshot_path='sentence2emb-model/models--sentence-transformers--paraphrase-multilingual-mpnet-base-v2/snapshots/ef15aed8b328d308d7237b9bf15269f2cd19e268'

In [36]:
!aws s3 cp --recursive {model_snapshot_path} s3://{bucket}/{s3_model_prefix}

upload: sentence2emb-model/models--sentence-transformers--paraphrase-multilingual-mpnet-base-v2/snapshots/ef15aed8b328d308d7237b9bf15269f2cd19e268/.gitattributes to s3://sagemaker-us-east-1-946277762357/LLM-RAG/workshop/sentence2emb-model/.gitattributes
upload: sentence2emb-model/models--sentence-transformers--paraphrase-multilingual-mpnet-base-v2/snapshots/ef15aed8b328d308d7237b9bf15269f2cd19e268/README.md to s3://sagemaker-us-east-1-946277762357/LLM-RAG/workshop/sentence2emb-model/README.md
upload: sentence2emb-model/models--sentence-transformers--paraphrase-multilingual-mpnet-base-v2/snapshots/ef15aed8b328d308d7237b9bf15269f2cd19e268/1_Pooling/config.json to s3://sagemaker-us-east-1-946277762357/LLM-RAG/workshop/sentence2emb-model/1_Pooling/config.json
upload: sentence2emb-model/models--sentence-transformers--paraphrase-multilingual-mpnet-base-v2/snapshots/ef15aed8b328d308d7237b9bf15269f2cd19e268/config_sentence_transformers.json to s3://sagemaker-us-east-1-946277762357/LLM-RAG/work

In [39]:
!pip install -U sagemaker

  Using cached sagemaker-2.147.0.tar.gz (718 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 1.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 1.7 MB/s eta 0:00:00
  Created wheel for sagemaker: filename=sagemaker-2.147.0-py2.py3-none-any.whl size=965152 sha256=982a75ec8fbd707d034a55c6e53f965fcb4ba9d43c568778b62f6f3b1d4cab93
  Stored in directory: /root/.cache/pip/wheels/6c/87/8b/f1cc496d34252d1accf573873b15d3b98d65aea92bca2f1e59
Successfully built sagemaker
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.132.0
    Uninstalling sagemaker-2.132.0:
      Successfully uninstalled sagemaker-2.132.0

[notice] A new release of pip is available: 23.0 -> 23.1
[notice] To update, run: pip install --upgrade pip


In [40]:
inference_image_uri = image_uris.retrieve(
    framework="djl-deepspeed", region=region, version="0.21.0"
)
print(f"Image going to be used is ---- > {inference_image_uri}")


Image going to be used is ---- > 763104351884.dkr.ecr.us-east-1.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.0-cu117


In [41]:
!mkdir -p sentence2emb_deploy_code

In [42]:
%%writefile sentence2emb_deploy_code/model.py
from djl_python import Input, Output
import torch
import logging
import math
import os
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, AutoModel

def load_model(properties):
    tensor_parallel = properties["tensor_parallel_degree"]
    model_location = properties['model_dir']
    if "model_id" in properties:
        model_location = properties['model_id']
    logging.info(f"Loading model in {model_location}")
    
    tokenizer = AutoTokenizer.from_pretrained(model_location)
   
    model = AutoModel.from_pretrained(
        model_location, 
        # device_map="balanced_low_0", 
        #load_in_8bit=True
    )
    model.requires_grad_(False)
    model.eval()
    
    return model, tokenizer


model = None
tokenizer = None
generator = None

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


def handle(inputs: Input):
    global model, tokenizer
    if not model:
        model, tokenizer = load_model(inputs.get_properties())

    if inputs.is_empty():
        return None
    data = inputs.get_as_json()
    
    input_sentences = data["inputs"]
    params = data["parameters"]
    logging.info(f"inputs: {input_sentences}")
    logging.info(f"parameters: {params}")
    
    encoded_input = tokenizer(input_sentences, padding=True, truncation=True, return_tensors='pt')
    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling. In this case, max pooling.
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask']).cpu().numpy()

#     # preprocess
#     input_ids = tokenizer(input_sentences, return_tensors="pt").input_ids
#     # pass inputs with all kwargs in data
#     if params is not None:
#         outputs = model.generate(input_ids, **params)
#     else:
#         outputs = model.generate(input_ids)

#     # postprocess the prediction
#     prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    result = {"sentence_embeddings": sentence_embeddings}
    return Output().add_as_json(result)

Writing sentence2emb_deploy_code/model.py


需要根据自己的账户更新S3url

In [43]:
%%writefile sentence2emb_deploy_code/serving.properties
engine=Python
option.tensor_parallel_degree=1
option.s3url = s3://sagemaker-us-east-1-946277762357/LLM-RAG/workshop/sentence2emb-model/

Writing sentence2emb_deploy_code/serving.properties


In [44]:
!rm s2e_model.tar.gz
!tar czvf s2e_model.tar.gz sentence2emb_deploy_code

rm: cannot remove 's2e_model.tar.gz': No such file or directory
sentence2emb_deploy_code/
sentence2emb_deploy_code/model.py
sentence2emb_deploy_code/serving.properties


In [45]:
s3_code_artifact = sess.upload_data("s2e_model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-us-east-1-946277762357/LLM-RAG/workshop/sentence2emb_deploy_code/s2e_model.tar.gz


In [46]:
from sagemaker.utils import name_from_base
import boto3

model_name = name_from_base(f"st-paraphrase-mpnet-base-v2") # Append a timestamp to the provided string
print(model_name)
print(f"Image going to be used is ---- > {inference_image_uri}")

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": inference_image_uri,
        "ModelDataUrl": s3_code_artifact
    },
    
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

st-paraphrase-mpnet-base-v2-2023-04-19-04-14-31-658
Image going to be used is ---- > 763104351884.dkr.ecr.us-east-1.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.0-cu117
Created Model: arn:aws:sagemaker:us-east-1:946277762357:model/st-paraphrase-mpnet-base-v2-2023-04-19-04-14-31-658


In [47]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g5.2xlarge",
            "InitialInstanceCount": 1,
            # "VolumeSizeInGB" : 400,
            # "ModelDataDownloadTimeoutInSeconds": 2400,
            "ContainerStartupHealthCheckTimeoutInSeconds": 15*60,
        },
    ],
)
endpoint_config_response

{'EndpointConfigArn': 'arn:aws:sagemaker:us-east-1:946277762357:endpoint-config/st-paraphrase-mpnet-base-v2-2023-04-19-04-14-31-658-config',
 'ResponseMetadata': {'RequestId': '3ec90224-74ef-4a10-9e8b-5e96fc786a1b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3ec90224-74ef-4a10-9e8b-5e96fc786a1b',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '139',
   'date': 'Wed, 19 Apr 2023 04:14:37 GMT'},
  'RetryAttempts': 0}}

In [48]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

Created Endpoint: arn:aws:sagemaker:us-east-1:946277762357:endpoint/st-paraphrase-mpnet-base-v2-2023-04-19-04-14-31-658-endpoint


In [49]:
import time

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:us-east-1:946277762357:endpoint/st-paraphrase-mpnet-base-v2-2023-04-19-04-14-31-658-endpoint
Status: InService


In [73]:
%%time
import json
import boto3

smr_client = boto3.client("sagemaker-runtime")

parameters = {
  #"early_stopping": True,
  #"length_penalty": 2.0,
  "max_new_tokens": 50,
  "temperature": 0,
  "min_length": 10,
  "no_repeat_ngram_size": 2,
}

CPU times: user 3.42 ms, sys: 0 ns, total: 3.42 ms
Wall time: 3.44 ms


In [74]:
prompts1 = """
请问AWS Clean Rooms是多方都会收费吗？
"""

response_model = smr_client.invoke_endpoint(
            EndpointName=endpoint_name,
            Body=json.dumps(
            {
                "inputs": [prompts1],
                "parameters": parameters
            }
            ),
            ContentType="application/json",
        )

response_model['Body'].read().decode('utf8')

'{\n  "sentence_embeddings":[\n    [\n      -0.0942641869187355,\n      0.13085055351257324,\n      -0.012362794950604439,\n      -0.02249862067401409,\n      -0.04928736388683319,\n      0.07633627206087112,\n      0.11948583275079727,\n      -0.048521898686885834,\n      0.031562961637973785,\n      -3.733066841959953e-05,\n      -0.07421016693115234,\n      -0.1545608639717102,\n      -0.00652194581925869,\n      0.025325052440166473,\n      -0.04009953886270523,\n      0.0862279087305069,\n      0.034070875495672226,\n      0.1100526824593544,\n      -0.03708256781101227,\n      -0.07843828201293945,\n      -0.08710747212171555,\n      -0.09370110929012299,\n      -0.042732104659080505,\n      -0.03769663721323013,\n      0.061826955527067184,\n      -0.11406314373016357,\n      -0.0506545826792717,\n      -0.05727188661694527,\n      0.16234733164310455,\n      0.10307776182889938,\n      0.22929474711418152,\n      0.10204465687274933,\n      0.08882841467857361,\n      0.1319449

In [63]:
!aws sagemaker delete-endpoint --endpoint-name st-paraphrase-mpnet-base-v2-2023-04-14-03-16-08-679-endpoint

In [64]:
!aws sagemaker delete-endpoint-config --endpoint-config-name st-paraphrase-mpnet-base-v2-2023-04-14-03-16-08-679-config